In [1]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Trabajo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import shutil
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alejandroczernikier/properati-argentina-dataset")

print("Path to dataset files:", path)

dest = "../data/"
os.makedirs(dest, exist_ok=True)
for file in os.listdir(path):
    shutil.move(os.path.join(path, file), os.path.join(dest, file))

100%|██████████| 227M/227M [00:09<00:00, 26.3MB/s] 

Extracting files...


Path to dataset files: C:\Users\Trabajo\.cache\kagglehub\datasets\alejandroczernikier\properati-argentina-dataset\versions\1


In [1]:
import pandas as pd
df = pd.read_csv("../data/entrenamiento.csv")

In [4]:
print(df.columns)

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'currency', 'price_period', 'title',
       'description', 'property_type', 'operation_type', 'price'],
      dtype='object')


creo un base de datos properati con dbeaver

In [ ]:
import pandas as pd

import sqlite3
import os


csv_path = "../data/entrenamiento.csv"
#db_path = os.path.join("../data","Properati.db")
folder_path = os.path.join("..", "data")
db_path = os.path.join(folder_path, "Properati.db")
table_name = "entranamiento"


def crear_base_datos():
    """Crea la base de datos SQLite y la pobla con los datos del torneo"""
    
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Crear tabla de jugadores
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS entrenamiento (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ad_type TEXT,
        start_date DATE,
        end_date DATE,
        created_on DATE,
        lat FLOAT,
        lon FLOAT,
        l1 TEXT,  
        l2 TEXT,
        l3 TEXT,
        l4 TEXT, 
        l5 TEXT,    
        l6 TEXT,
        rooms INTEGER,
        bedrooms INTEGER,
        bathrooms INTEGER,  
        surface_total FLOAT,
        surface_covered FLOAT,
        currency TEXT,
        price_period TEXT,
        title TEXT,
        description TEXT,
        property_type TEXT,
        operation_type TEXT,
        price FLOAT
    )
    ''')
    #['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
    #   'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
    #   'surface_total', 'surface_covered', 'currency', 'price_period', 'title',
    #   'description', 'property_type', 'operation_type', 'price']
    
    # Leer datos del CSV
    df = pd.read_csv(csv_path)  
    
    
    datos_propiedades = []
    
    
    # Insertar datos
    cursor.executemany('''
    INSERT INTO entrenamiento (
    ad_type, start_date, end_date, created_on, lat, lon,
    l1, l2, l3, l4, l5, l6,
    rooms, bedrooms, bathrooms,
    surface_total, surface_covered,
    currency, price_period,
    title, description,
    property_type, operation_type, price
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', datos_propiedades)
    
    
    
    conn.commit()

    conn.close()

if not(os.path.exists(db_path)):
    crear_base_datos()